In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
from pyspark import SparkFiles

# Create a SparkSession
spark = SparkSession.builder.appName("TempViews").getOrCreate()

In [3]:
# Read the data from the csv on GitHub
from pyspark import SparkFiles
url = 'https://raw.githubusercontent.com/DKogge/project4/main/Resources/bank-additional-full.csv'
spark.sparkContext.addFile(url)


bank_additional_full = spark.read.option('header', 'true').csv(SparkFiles.get("bank-additional-full.csv"), inferSchema=True, sep=';', timestampFormat="mm/dd/yy")

# Show DataFrame
bank_additional_full.show()


+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|  housemaid| married|           basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57|   services| married|        high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.

In [4]:
 # Print our schema
bank_additional_full.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- y: string (nullable = true)



In [5]:
# Fill Missing Values
bank_additional_filled = bank_additional_full.na.fill(0)

# Show DataFrame
bank_additional_filled.show()


+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|  housemaid| married|           basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57|   services| married|        high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.

In [6]:
bank_additional_filled.count()

41188

In [7]:
bank_additional_filled.createOrReplaceTempView('bank_additional_filled')

In [8]:
# To get the data for just Job, marital status and Education
query = """
SELECT job, marital, education
from bank_additional_filled
"""
spark.sql(query).show()

+-----------+--------+-------------------+
|        job| marital|          education|
+-----------+--------+-------------------+
|  housemaid| married|           basic.4y|
|   services| married|        high.school|
|   services| married|        high.school|
|     admin.| married|           basic.6y|
|   services| married|        high.school|
|   services| married|           basic.9y|
|     admin.| married|professional.course|
|blue-collar| married|            unknown|
| technician|  single|professional.course|
|   services|  single|        high.school|
|blue-collar| married|            unknown|
|   services|  single|        high.school|
|blue-collar|  single|        high.school|
|  housemaid|divorced|           basic.4y|
|blue-collar| married|           basic.6y|
|    retired| married|           basic.9y|
|blue-collar| married|           basic.6y|
|blue-collar| married|           basic.6y|
|blue-collar| married|           basic.9y|
| management|  single|           basic.9y|
+----------

In [9]:
# Get the count of customers per job category:
query ="""
SELECT job, COUNT(*) as count
FROM bank_additional_filled
GROUP BY job;
"""
spark.sql(query).show()

+-------------+-----+
|          job|count|
+-------------+-----+
|   management| 2924|
|      retired| 1720|
|      unknown|  330|
|self-employed| 1421|
|      student|  875|
|  blue-collar| 9254|
| entrepreneur| 1456|
|       admin.|10422|
|   technician| 6743|
|     services| 3969|
|    housemaid| 1060|
|   unemployed| 1014|
+-------------+-----+



In [10]:
# Find the average age of customers by marital status
query ="""
SELECT marital, AVG(age) as avg_age
FROM bank_additional_filled
GROUP BY marital;
"""
spark.sql(query).show()

+--------+-----------------+
| marital|          avg_age|
+--------+-----------------+
| unknown|           40.275|
|divorced|44.89939288811795|
| married|42.30716463414634|
|  single|33.15871369294606|
+--------+-----------------+



In [11]:
# Calls by day of the week
query="""
SELECT day_of_week, COUNT(*) AS number_of_calls
FROM bank_additional_filled
GROUP BY day_of_week
order by
case day_of_week
when 'mon' then 1
when 'tue' then 2
when 'wed' then 3
when 'thu' then 4
when 'fri' then 5 end;
"""
spark.sql(query).show()

+-----------+---------------+
|day_of_week|number_of_calls|
+-----------+---------------+
|        mon|           8514|
|        tue|           8090|
|        wed|           8134|
|        thu|           8623|
|        fri|           7827|
+-----------+---------------+



In [12]:
# Get the number of customers contacted during different months
query ="""
SELECT month, COUNT(*) as count
FROM bank_additional_filled
GROUP BY month;
"""
spark.sql(query).show()

+-----+-----+
|month|count|
+-----+-----+
|  jun| 5318|
|  aug| 6178|
|  may|13769|
|  mar|  546|
|  oct|  718|
|  jul| 7174|
|  nov| 4101|
|  apr| 2632|
|  dec|  182|
|  sep|  570|
+-----+-----+



In [13]:
# Rename the default column to `default credit` and show only this new column
df = bank_additional_filled
df = df.withColumnRenamed('default', 'default credit')
df.show()

+---+-----------+--------+-------------------+--------------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|        job| marital|          education|default credit|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+-----------+--------+-------------------+--------------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|  housemaid| married|           basic.4y|            no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57|   services| married|        high.school|       unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|n

In [14]:
# Converting Pyspark DataFRame to Pandas DataFrame
import pandas as pd
pd.set_option('display.max_columns', None)
bank_marketing_df = df.toPandas()

#Show the DataFframe
bank_marketing_df.head()

,age,job,marital,education,default credit,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [15]:
#Save the Dataframe to a csvFile
bank_marketing_df.to_csv('output.csv', index=False)

In [16]:
# Determine the number of unique values in each column.
bank_marketing_df.nunique()

age                 78
job                 12
marital              4
education            8
default credit       3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

In [17]:
bank_marketing_df = bank_marketing_df.drop(columns=['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m'])
bank_marketing_df.head()

,age,job,marital,education,default credit,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,5191.0,no


In [18]:
 # Check the Job value counts to identify and replace with "Other"
bank_marketing_df['job'].value_counts()

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64

In [19]:
# Choose a cutoff value and create a list of jobs to be replaced
# use the variable name `job_to_replace`
value_counts = bank_marketing_df['job'].value_counts()
job_to_replace = value_counts[value_counts < 1050].index

# Replace in dataframe
for jobs in job_to_replace:
  bank_marketing_df['job'] = bank_marketing_df['job'].replace(jobs,"Other")

# Check to make sure replacement was successful
bank_marketing_df['job'].value_counts()

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
Other             2219
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
Name: count, dtype: int64

In [20]:
# Check the value_counts
bank_marketing_df['marital'].value_counts()

marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64

In [21]:
# Check the value_counts
bank_marketing_df['education'].value_counts()

education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64

In [22]:
# Choose a cutoff value and create a list of education values to be replaced
# use the variable name `edu_to_replace`
value_counts = bank_marketing_df['education'].value_counts()
edu_to_replace = value_counts[value_counts < 2000].index

# Replace in dataframe
for e in edu_to_replace:
  bank_marketing_df['education'] = bank_marketing_df['education'].replace(e,"Other")

# Check to make sure replacement was successful
bank_marketing_df['education'].value_counts()

education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
Other                   1749
Name: count, dtype: int64

In [23]:
# Generate our categorical variable lists and check the number of unique values in each column
bank_marketing_cat = bank_marketing_df.dtypes[bank_marketing_df.dtypes == "object"].index.tolist()
bank_marketing_df[bank_marketing_cat].nunique()

job               10
marital            4
education          7
default credit     3
housing            3
loan               3
contact            2
month             10
day_of_week        5
poutcome           3
y                  2
dtype: int64

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
bank_marketing_dummies_df = pd.get_dummies(bank_marketing_df, columns=['job', 'marital', 'education', 'default credit', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y'], drop_first=True)
bank_marketing_dummies_df = bank_marketing_dummies_df.astype(int)
bank_marketing_dummies_df.head()

,age,duration,campaign,pdays,previous,nr.employed,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_technician,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_professional.course,education_university.degree,default credit_unknown,default credit_yes,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success,y_yes
0,56,261,1,999,0,5191,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
1,57,149,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
2,37,226,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
3,40,151,1,999,0,5191,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
4,56,307,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0


In [25]:
#shorten DF name
bank_df = bank_marketing_dummies_df.copy()
bank_df.head()

,age,duration,campaign,pdays,previous,nr.employed,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_technician,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_professional.course,education_university.degree,default credit_unknown,default credit_yes,housing_unknown,housing_yes,loan_unknown,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success,y_yes
0,56,261,1,999,0,5191,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
1,57,149,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
2,37,226,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
3,40,151,1,999,0,5191,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
4,56,307,1,999,0,5191,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0


In [26]:
# Import more dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [27]:
# Split our preprocessed data into our features and target arrays
y = bank_df["y_yes"]
X = bank_df.drop(columns="y_yes")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [28]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
# Define the model  *** first iteration to set a baseline ***
number_input_features = (X_train.shape[1])
hidden_nodes_layer1 =  5
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 235       
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 271 (1.06 KB)
Trainable params: 271 (1.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
966/966 [==============================] - 4s 3ms/step - loss: 0.3486 - accuracy: 0.8779
Epoch 2/100
966/966 [==============================] - 2s 2ms/step - loss: 0.2031 - accuracy: 0.9074
Epoch 3/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1925 - accuracy: 0.9090
Epoch 4/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1894 - accuracy: 0.9089
Epoch 5/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1879 - accuracy: 0.9095
Epoch 6/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1872 - accuracy: 0.9094
Epoch 7/100
966/966 [==============================] - 3s 3ms/step - loss: 0.1865 - accuracy: 0.9097
Epoch 8/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1861 - accuracy: 0.9104
Epoch 9/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1856 - accuracy: 0.9103
Epoch 10/100
966/966 [==============================] - 2s 2ms/step - loss: 0.1851 - accura

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 1s - loss: 0.1847 - accuracy: 0.9145 - 588ms/epoch - 2ms/step
Loss: 0.1846647411584854, Accuracy: 0.9145382046699524


In [33]:
# Export our model to HDF5 file
nn.save("optimizationedu.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
